In [78]:
!cat container/Dockerfile


# Copyright 2017-2018 Amazon.com, Inc. or its affiliates. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License"). You
# may not use this file except in compliance with the License. A copy of
# the License is located at
#
#     http://aws.amazon.com/apache2.0/
#
# or in the "license" file accompanying this file. This file is
# distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF
# ANY KIND, either express or implied. See the License for the specific
# language governing permissions and limitations under the License.

# For more information on creating a Dockerfile
# https://docs.docker.com/compose/gettingstarted/#step-2-create-a-dockerfile
# https://github.com/awslabs/amazon-sagemaker-examples/master/advanced_functionality/pytorch_extending_our_containers/pytorch_extending_our_containers.ipynb
ARG REGION=ap-southeast-1

# SageMaker PyTorch image
FROM 763104351884.dkr.ecr.ap-southeast-1.amazonaws.com/pytorch-training:1.6.0-cpu-py36-ubuntu1

In [10]:
!pip install sagemaker-studio-image-build

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


In [84]:
%%sh

cd container

sm-docker build .  --repository pytorch-bring-your-own-catainer:latest
# sm-docker build .  --repository rnn-forecast:latest

....[Container] 2022/02/23 13:48:50 Waiting for agent ping

[Container] 2022/02/23 13:48:51 Waiting for DOWNLOAD_SOURCE
[Container] 2022/02/23 13:48:54 Phase is DOWNLOAD_SOURCE
[Container] 2022/02/23 13:48:54 CODEBUILD_SRC_DIR=/codebuild/output/src304463565/src
[Container] 2022/02/23 13:48:54 YAML location is /codebuild/output/src304463565/src/buildspec.yml
[Container] 2022/02/23 13:48:54 Setting HTTP client timeout to higher timeout for S3 source
[Container] 2022/02/23 13:48:54 Processing environment variables
[Container] 2022/02/23 13:48:54 No runtime version selected in buildspec.
[Container] 2022/02/23 13:48:54 Moving to directory /codebuild/output/src304463565/src
[Container] 2022/02/23 13:48:54 Configuring ssm agent with target id: codebuild:9f0a3f52-0bab-47de-8a15-e5be9db30e11
[Container] 2022/02/23 13:48:54 Successfully updated ssm agent configuration
[Container] 2022/02/23 13:48:54 Registering with agent
[Container] 2022/02/23 13:48:54 Phases found in YAML: 3
[Container] 2022/

In [32]:
from sagemaker import get_execution_role

role = get_execution_role()

In [85]:
from sagemaker.estimator import Estimator

hyperparameters = {"epochs": 100}

estimator = Estimator(
    role=role,
    train_instance_count=1,
    train_instance_type="ml.c5.xlarge",
    image_uri="205124155157.dkr.ecr.ap-southeast-1.amazonaws.com/pytorch-bring-your-own-catainer:latest",
    hyperparameters=hyperparameters,
)
estimator.fit({'train': 's3://sagemaker-ap-southeast-1-205124155157/data'})


train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


2022-02-23 13:57:23 Starting - Starting the training job...
2022-02-23 13:57:38 Starting - Preparing the instances for trainingProfilerReport-1645624643: InProgress
......
2022-02-23 13:58:50 Downloading - Downloading input data...
2022-02-23 13:59:20 Training - Downloading the training image......
2022-02-23 14:00:21 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-02-23 14:00:23,610 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-02-23 14:00:23,612 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-02-23 14:00:23,621 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-02-23 14:00:23,627 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-02-23 14:00:23,629 sagemaker-training-toolkit INFO     No G

In [170]:
model_path=estimator.model_data

In [171]:
model_path

's3://sagemaker-ap-southeast-1-205124155157/pytorch-bring-your-own-catainer-2022-02-23-13-57-22-574/output/model.tar.gz'

In [230]:
from sagemaker.pytorch import PyTorchModel

pytorch_model = PyTorchModel(model_data=model_path, 
                             role=role, 
                             entry_point='inference/inference.py', 
                             framework_version="1.8.1",
                             py_version='py3',
)
predictor = pytorch_model.deploy(instance_type='ml.t2.medium', initial_instance_count=1)

--------!

In [231]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()

role = sagemaker.get_execution_role()
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('./data/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
filepath = './data/stockprices.csv'
data = pd.read_csv(filepath)
# data = data.sort_values('Date')

./data/UNH_2006-01-01_to_2018-01-01.csv
./data/PFE_2006-01-01_to_2018-01-01.csv
./data/BA_2006-01-01_to_2018-01-01.csv
./data/stockprices.csv
./data/all_stocks_2006-01-01_to_2018-01-01.csv
./data/PG_2006-01-01_to_2018-01-01.csv
./data/MMM_2006-01-01_to_2018-01-01.csv
./data/JPM_2006-01-01_to_2018-01-01.csv
./data/AABA_2006-01-01_to_2018-01-01.csv
./data/CVX_2006-01-01_to_2018-01-01.csv
./data/all_stocks_2017-01-01_to_2018-01-01.csv
./data/UTX_2006-01-01_to_2018-01-01.csv
./data/AXP_2006-01-01_to_2018-01-01.csv
./data/GE_2006-01-01_to_2018-01-01.csv
./data/TRV_2006-01-01_to_2018-01-01.csv
./data/HD_2006-01-01_to_2018-01-01.csv
./data/MCD_2006-01-01_to_2018-01-01.csv
./data/XOM_2006-01-01_to_2018-01-01.csv
./data/VZ_2006-01-01_to_2018-01-01.csv
./data/DIS_2006-01-01_to_2018-01-01.csv
./data/CAT_2006-01-01_to_2018-01-01.csv
./data/CSCO_2006-01-01_to_2018-01-01.csv
./data/WMT_2006-01-01_to_2018-01-01.csv
./data/GOOGL_2006-01-01_to_2018-01-01.csv
./data/IBM_2006-01-01_to_2018-01-01.csv
./da

In [232]:
import matplotlib.pyplot as plt
# !pip install seaborn
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
price = data[['Close']]

scaler = MinMaxScaler(feature_range=(-1, 1))
price['Close'] = scaler.fit_transform(price['Close'].values.reshape(-1,1))

def split_data(stock, lookback):
    data_raw = stock.to_numpy() # convert to numpy array
    data = []

    # create all possible sequences of length seq_len
    for index in range(len(data_raw) - lookback): 

        data.append(data_raw[index: index + lookback])
        
    data = np.array(data);
    test_set_size = int(np.round(0.2*data.shape[0]));
    train_set_size = data.shape[0] - (test_set_size);

    x_train = data[:train_set_size,:-1,:]
    y_train = data[:train_set_size,-1,:]
    
    x_test = data[train_set_size:,:-1]
    y_test = data[train_set_size:,-1,:]
    
    return [x_train, y_train, x_test, y_test]
lookback = 5 # choose sequence length
x_train, y_train, x_test, y_test = split_data(price, lookback)
print('x_train.shape = ',x_train.shape)
print('y_train.shape = ',y_train.shape)
print('x_test.shape = ',x_test.shape)
print('y_test.shape = ',y_test.shape)
price.to_numpy()[0:5]

x_train.shape =  (1406, 4, 1)
y_train.shape =  (1406, 1)
x_test.shape =  (351, 4, 1)
y_test.shape =  (351, 1)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



array([[-0.91248866],
       [-0.92933727],
       [-0.9320509 ],
       [-0.94002002],
       [-0.90824265]])

In [233]:
import torch
import torch.nn as nn

class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.fc(out[:, -1, :]) 
        return out

In [234]:
import torch
import torch.nn as nn

x_train = torch.from_numpy(x_train).type(torch.Tensor)
x_test = torch.from_numpy(x_test).type(torch.Tensor)
y_train_lstm = torch.from_numpy(y_train).type(torch.Tensor)
y_test_lstm = torch.from_numpy(y_test).type(torch.Tensor)
y_train_gru = torch.from_numpy(y_train).type(torch.Tensor)
y_test_gru = torch.from_numpy(y_test).type(torch.Tensor)


In [235]:
input_dim = 1
hidden_dim = 32
num_layers = 2
output_dim = 1
num_epochs = 100
model = LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)
criterion = torch.nn.MSELoss(reduction='mean')
optimiser = torch.optim.Adam(model.parameters(), lr=0.01)

In [236]:
import time

hist = np.zeros(num_epochs)
start_time = time.time()
lstm = []

for t in range(num_epochs):
    y_train_pred = model(x_train)

    loss = criterion(y_train_pred, y_train_lstm)
    print("Epoch ", t, "MSE: ", loss.item())
    hist[t] = loss.item()

    optimiser.zero_grad()
    loss.backward()
    optimiser.step()
    
training_time = time.time()-start_time
print("Training time: {}".format(training_time))

Epoch  0 MSE:  0.60541832447052
Epoch  1 MSE:  0.4530981779098511
Epoch  2 MSE:  0.316417932510376
Epoch  3 MSE:  0.17949502170085907
Epoch  4 MSE:  0.06407473981380463
Epoch  5 MSE:  0.08606742322444916
Epoch  6 MSE:  0.14131617546081543
Epoch  7 MSE:  0.09448736906051636
Epoch  8 MSE:  0.05262463167309761
Epoch  9 MSE:  0.04773187264800072
Epoch  10 MSE:  0.0613354854285717
Epoch  11 MSE:  0.07423778623342514
Epoch  12 MSE:  0.07887696474790573
Epoch  13 MSE:  0.07508771121501923
Epoch  14 MSE:  0.06572586297988892
Epoch  15 MSE:  0.0547044612467289
Epoch  16 MSE:  0.045999184250831604
Epoch  17 MSE:  0.042527712881565094
Epoch  18 MSE:  0.04472038894891739
Epoch  19 MSE:  0.04976769536733627
Epoch  20 MSE:  0.05313155800104141
Epoch  21 MSE:  0.05193532258272171
Epoch  22 MSE:  0.046920761466026306
Epoch  23 MSE:  0.04111208766698837
Epoch  24 MSE:  0.03717750683426857
Epoch  25 MSE:  0.035951707512140274
Epoch  26 MSE:  0.036582883447408676
Epoch  27 MSE:  0.03746436908841133
Epoch

In [237]:
predict = pd.DataFrame(scaler.inverse_transform(y_train_pred.detach().numpy()))
original = pd.DataFrame(scaler.inverse_transform(y_train_lstm.detach().numpy()))


In [238]:
import math, time
from sklearn.metrics import mean_squared_error

# make predictions
y_test_pred = predictor.predict(x_test)

# invert predictions
y_train_pred = scaler.inverse_transform(y_train_pred.detach().numpy())
y_train = scaler.inverse_transform(y_train_lstm.detach().numpy())
y_test_pred = scaler.inverse_transform(y_test_pred)
y_test = scaler.inverse_transform(y_test_lstm.detach().numpy())

# calculate root mean squared error
lstm = []

trainScore = math.sqrt(mean_squared_error(y_train[:,0], y_train_pred[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(y_test[:,0], y_test_pred[:,0]))
print('Test Score: %.2f RMSE' % (testScore))
lstm.append(trainScore)
lstm.append(testScore)

Train Score: 10.46 RMSE
Test Score: 171.04 RMSE


In [108]:
# shift train predictions for plotting
trainPredictPlot = np.empty_like(price)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[lookback:len(y_train_pred)+lookback, :] = y_train_pred

# shift test predictions for plotting
testPredictPlot = np.empty_like(price)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(y_train_pred)+lookback-1:len(price)-1, :] = y_test_pred

original = scaler.inverse_transform(price['Close'].values.reshape(-1,1))

predictions = np.append(trainPredictPlot, testPredictPlot, axis=1)
predictions = np.append(predictions, original, axis=1)
result = pd.DataFrame(predictions)

In [109]:
# !pip install nbformat
# !pip install plotly
import plotly.express as px
import plotly.graph_objects as go

fig = go.Figure()
# fig.add_trace(go.Scatter(go.Scatter(x=result.index, y=result[0],
#                     mode='lines',
#                     name='Train prediction')))
fig.add_trace(go.Scatter(x=result.index, y=result[1],
                    mode='lines',
                    name='Test prediction'))
fig.add_trace(go.Scatter(go.Scatter(x=result.index, y=result[2],
                    mode='lines',
                    name='Actual Value')))
fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=True,
        showticklabels=False,
        linecolor='white',
        linewidth=2
    ),
    yaxis=dict(
        title_text='Close (USD)',
        titlefont=dict(
            family='Rockwell',
            size=12,
            color='white',
        ),
        showline=True,
        showgrid=True,
        showticklabels=True,
        linecolor='white',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Rockwell',
            size=12,
            color='white',
        ),
    ),
    showlegend=True,
    template = 'plotly_dark'

)



annotations = []
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='Results (LSTM)',
                              font=dict(family='Rockwell',
                                        size=26,
                                        color='white'),
                              showarrow=False))
fig.update_layout(annotations=annotations)


In [110]:
!pip install chart-studio
!pip install nbformat
!pip install plotly

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.
/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrad

In [111]:
import chart_studio.plotly as py
import chart_studio

chart_studio.tools.set_credentials_file(username='awsryanlee', api_key='n9L15EdJiAQ6EAf2blmi')
py.iplot(fig, filename='stock_prediction_lstm')